# 02 — Tax-aware rebalance (snapshot)
Demonstrates tax-lot selling logic with public prices.

In [ ]:
from wealth_lab.config import DEFAULT_UNIVERSE
from wealth_lab.data.prices_yf import fetch_adj_close
from wealth_lab.tax_rebal.tax_aware import (
    compute_rebalance_trades,
    demo_make_random_lots,
    choose_lots_to_sell,
    estimate_tax_liability,
    TaxConfig,
)

tickers = list(DEFAULT_UNIVERSE.tickers)
px = fetch_adj_close(tickers, start="2020-01-01")
prices = px.resample("M").last().iloc[-1].to_dict()

nav = 1_000_000
# start from equal-weight holdings
holdings = {t: (nav/len(tickers))/float(prices[t]) for t in tickers}

# pretend target weights came from an optimiser
target_weights = {t: w for t, w in zip(tickers, [0.35, 0.25, 0.15, 0.15, 0.10])}

trades = compute_rebalance_trades(holdings, prices, target_weights, nav)
qty_to_sell = {a: -q for a, q in trades.items() if q < 0}

lots = demo_make_random_lots(tickers, holdings, prices, seed=42)
sells = choose_lots_to_sell(lots, prices, qty_to_sell)

tax = estimate_tax_liability(sells, TaxConfig(cg_rate=0.20))
tax


In [ ]:
import pandas as pd
table = pd.DataFrame([{
    "asset": a,
    "sell_qty": q,
    "price": pxu,
    "cost_basis": lot.cost,
    "gain_per_unit": (pxu - lot.cost),
} for a, lot, q, pxu in sells])
table.head(20)
